In [ ]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

Using TensorFlow backend.


In [ ]:
def data():
    # Split train & validation set
    DDD_Train_Decile3 = pd.read_pickle("DDD_Train_Decile3.pkl")
    DDD_Train_Targets_1D = pd.read_pickle("DDD_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("DDD_Test_Decile3.pkl")
    y_test = pd.read_pickle("DDD_Test_Targets_1D.pkl")
    
    X_train, X_val, y_train, y_val = train_test_split(DDD_Train_Decile3, DDD_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones',
                moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None,
                beta_constraint=None, gamma_constraint=None))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,75,100,125,150])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM DDD Decile 3')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

2264/2264 [==============================] - 5s 2ms/step - loss: 0.4743 - acc: 0.7875 - val_loss: 0.4606 - val_acc: 0.7954
Epoch 36/75
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4717 - acc: 0.7955 - val_loss: 0.4595 - val_acc: 0.7937
Epoch 37/75
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4765 - acc: 0.7840 - val_loss: 0.4591 - val_acc: 0.7937
Epoch 38/75
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4705 - acc: 0.7946 - val_loss: 0.4585 - val_acc: 0.7937
Epoch 39/75
2264/2264 [==============================] - 6s 2ms/step - loss: 0.4645 - acc: 0.7937 - val_loss: 0.4590 - val_acc: 0.7972
Epoch 40/75
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4747 - acc: 0.7893 - val_loss: 0.4584 - val_acc: 0.7972
Epoch 41/75
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4665 - acc: 0.7920 - val_loss: 0.4572 - val_acc: 0.7937
Epoch 42/75
2264/2264 [==============================] - 5s 2ms/ste

Epoch 20/75
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4388 - acc: 0.7920 - val_loss: 0.4404 - val_acc: 0.7937
Epoch 21/75
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4457 - acc: 0.7942 - val_loss: 0.4434 - val_acc: 0.8042
Epoch 22/75
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4511 - acc: 0.7884 - val_loss: 0.4400 - val_acc: 0.7937

Epoch 00022: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 23/75
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4404 - acc: 0.7999 - val_loss: 0.4393 - val_acc: 0.7901
Epoch 24/75
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4410 - acc: 0.7964 - val_loss: 0.4400 - val_acc: 0.7901
Epoch 25/75
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4363 - acc: 0.7986 - val_loss: 0.4410 - val_acc: 0.7884
Epoch 26/75
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4388 - acc: 0.7968 - val_loss

2264/2264 [==============================] - 6s 2ms/step - loss: 0.5438 - acc: 0.7522 - val_loss: 0.4961 - val_acc: 0.7795
Epoch 4/100
2264/2264 [==============================] - 6s 2ms/step - loss: 0.5018 - acc: 0.7805 - val_loss: 0.4732 - val_acc: 0.7972
Epoch 5/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4902 - acc: 0.7849 - val_loss: 0.4698 - val_acc: 0.7937
Epoch 6/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4808 - acc: 0.7973 - val_loss: 0.4649 - val_acc: 0.7954
Epoch 7/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4801 - acc: 0.7853 - val_loss: 0.4609 - val_acc: 0.8025
Epoch 8/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4733 - acc: 0.7924 - val_loss: 0.4581 - val_acc: 0.7989
Epoch 9/100
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4746 - acc: 0.7898 - val_loss: 0.4561 - val_acc: 0.7989
Epoch 10/100
2264/2264 [==============================] - 4s 2ms/st

2264/2264 [==============================] - 6s 3ms/step - loss: 0.4206 - acc: 0.8030 - val_loss: 0.4324 - val_acc: 0.8025
Epoch 89/125
2264/2264 [==============================] - 6s 3ms/step - loss: 0.4251 - acc: 0.8026 - val_loss: 0.4323 - val_acc: 0.8042
Epoch 90/125
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4210 - acc: 0.8048 - val_loss: 0.4319 - val_acc: 0.8025
Epoch 91/125
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4260 - acc: 0.8012 - val_loss: 0.4312 - val_acc: 0.8007
Epoch 92/125
2264/2264 [==============================] - 6s 3ms/step - loss: 0.4291 - acc: 0.8065 - val_loss: 0.4313 - val_acc: 0.8007
Epoch 93/125
2264/2264 [==============================] - 6s 2ms/step - loss: 0.4297 - acc: 0.7973 - val_loss: 0.4309 - val_acc: 0.8007
Epoch 94/125
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4291 - acc: 0.8004 - val_loss: 0.4310 - val_acc: 0.8007
Epoch 95/125
2264/2264 [==============================] - 6s 

Epoch 86/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4675 - acc: 0.7915 - val_loss: 0.4576 - val_acc: 0.7919
Epoch 87/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4677 - acc: 0.7928 - val_loss: 0.4576 - val_acc: 0.7919
Epoch 88/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4771 - acc: 0.7853 - val_loss: 0.4574 - val_acc: 0.7919
Epoch 89/100
2264/2264 [==============================] - 6s 3ms/step - loss: 0.4766 - acc: 0.7862 - val_loss: 0.4574 - val_acc: 0.7919
Epoch 90/100
2264/2264 [==============================] - 6s 3ms/step - loss: 0.4657 - acc: 0.7924 - val_loss: 0.4575 - val_acc: 0.7919
Epoch 91/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4732 - acc: 0.7924 - val_loss: 0.4573 - val_acc: 0.7901
Epoch 92/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4704 - acc: 0.7906 - val_loss: 0.4575 - val_acc: 0.7919
Epoch 93/100
2264/2264 [========================

2264/2264 [==============================] - 5s 2ms/step - loss: 0.4684 - acc: 0.7902 - val_loss: 0.4623 - val_acc: 0.7919
Epoch 51/125
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4674 - acc: 0.7898 - val_loss: 0.4616 - val_acc: 0.7919
Epoch 52/125
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4722 - acc: 0.7884 - val_loss: 0.4617 - val_acc: 0.7937
Epoch 53/125
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4673 - acc: 0.7893 - val_loss: 0.4618 - val_acc: 0.7937
Epoch 54/125
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4633 - acc: 0.7977 - val_loss: 0.4619 - val_acc: 0.7954
Epoch 55/125
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4577 - acc: 0.7924 - val_loss: 0.4606 - val_acc: 0.7972
Epoch 56/125
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4710 - acc: 0.7920 - val_loss: 0.4612 - val_acc: 0.7972

Epoch 00056: ReduceLROnPlateau reducing learning rate to 1e-

Epoch 110/125
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4722 - acc: 0.7902 - val_loss: 0.4590 - val_acc: 0.7954
Epoch 111/125
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4628 - acc: 0.7968 - val_loss: 0.4590 - val_acc: 0.7954
Epoch 112/125
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4667 - acc: 0.7880 - val_loss: 0.4586 - val_acc: 0.7954
Epoch 113/125
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4599 - acc: 0.7937 - val_loss: 0.4588 - val_acc: 0.7954
Epoch 114/125
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4619 - acc: 0.7933 - val_loss: 0.4586 - val_acc: 0.7954
Epoch 115/125
2264/2264 [==============================] - 6s 3ms/step - loss: 0.4596 - acc: 0.7906 - val_loss: 0.4586 - val_acc: 0.7937
Epoch 116/125
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4735 - acc: 0.7933 - val_loss: 0.4585 - val_acc: 0.7954
Epoch 117/125
2264/2264 [================

Epoch 43/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4371 - acc: 0.8004 - val_loss: 0.4389 - val_acc: 0.8060
Epoch 44/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4366 - acc: 0.7977 - val_loss: 0.4388 - val_acc: 0.8060
Epoch 45/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4399 - acc: 0.8048 - val_loss: 0.4387 - val_acc: 0.8060
Epoch 46/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4263 - acc: 0.8065 - val_loss: 0.4388 - val_acc: 0.8060
Epoch 47/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4301 - acc: 0.8021 - val_loss: 0.4390 - val_acc: 0.8060
Epoch 48/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4335 - acc: 0.8048 - val_loss: 0.4391 - val_acc: 0.8060
Epoch 49/100
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4303 - acc: 0.8079 - val_loss: 0.4391 - val_acc: 0.8060
Epoch 50/100
2264/2264 [========================

2264/2264 [==============================] - 4s 2ms/step - loss: 0.4318 - acc: 0.7999 - val_loss: 0.4384 - val_acc: 0.7937
Epoch 43/50
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4390 - acc: 0.8061 - val_loss: 0.4384 - val_acc: 0.7937
Epoch 44/50
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4369 - acc: 0.8114 - val_loss: 0.4384 - val_acc: 0.7937
Epoch 45/50
2264/2264 [==============================] - 4s 2ms/step - loss: 0.4362 - acc: 0.8030 - val_loss: 0.4382 - val_acc: 0.7954
Epoch 46/50
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4434 - acc: 0.7977 - val_loss: 0.4382 - val_acc: 0.7954
Epoch 47/50
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4414 - acc: 0.7946 - val_loss: 0.4385 - val_acc: 0.7937
Epoch 48/50
2264/2264 [==============================] - 5s 2ms/step - loss: 0.4427 - acc: 0.8039 - val_loss: 0.4384 - val_acc: 0.7937
Epoch 49/50
2264/2264 [==============================] - 5s 2ms/ste

In [ ]:
0.86434